In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm
%matplotlib inline
pd.options.display.max_rows = 999

In [2]:
accre_jobs = pd.read_csv('../data/accre-jobs-2020.csv', nrows=2)

In [3]:
columns_list= accre_jobs.columns.to_list()
columns_list

['JOBID',
 'ACCOUNT',
 'USER',
 'REQMEM',
 'USEDMEM',
 'REQTIME',
 'USEDTIME',
 'NODES',
 'CPUS',
 'PARTITION',
 'EXITCODE',
 'STATE',
 'NODELIST']

In [4]:
accre_jobs

,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,STATE,NODELIST
0,15925210,treviso,arabella,122880Mn,65973.49M,13-18:00:00,13-18:00:28,1,24,production,0:0,COMPLETED,cn1531
1,15861126,treviso,arabella,122880Mn,67181.12M,13-18:00:00,12-14:50:56,1,24,production,0:0,COMPLETED,cn1441


In [5]:
accre_jobs = pd.read_csv('../data/accre-jobs-2020.csv', nrows=2, usecols=columns_list)

In [6]:
accre_jobs = pd.read_csv('../data/accre-jobs-2020.csv', nrows=3500, usecols=columns_list)

In [7]:
accre_jobs.loc[3460:3470]

,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,STATE,NODELIST
3460,17050901_91,winged,lavonda,4096Mn,669.61M,12:00:00,00:06:05,4,1,production,0:0,COMPLETED,cn[449
3461,17050901_92,winged,lavonda,4096Mn,500.06M,12:00:00,00:02:25,1,4,production,0:0,COMPLETED,cn1336
3462,17050901_93,winged,lavonda,4096Mn,530.52M,12:00:00,00:02:19,1,4,production,0:0,COMPLETED,cn1319
3463,17050901_94,winged,lavonda,4096Mn,620.31M,12:00:00,00:05:59,3,1,production,0:0,COMPLETED,cn[425
3464,17050901_95,winged,lavonda,4096Mn,484.49M,12:00:00,00:02:08,1,4,production,0:0,COMPLETED,cn1319
3465,17050901_96,winged,lavonda,4096Mn,304.21M,12:00:00,00:02:16,1,4,production,0:0,COMPLETED,cn1336
3466,17050901_97,winged,lavonda,4096Mn,629.38M,12:00:00,00:06:24,4,1,production,0:0,COMPLETED,cn[394
3467,17050901_98,winged,lavonda,4096Mn,561.52M,12:00:00,00:02:04,1,4,production,0:0,COMPLETED,cn1319
3468,17050901_99,winged,lavonda,4096Mn,636.64M,12:00:00,00:02:00,1,4,production,0:0,COMPLETED,cn1319
3469,17050901_100,winged,lavonda,4096Mn,522.07M,12:00:00,00:02:09,1,4,production,0:0,COMPLETED,cn1336


In [8]:
with open('../data/accre-jobs-2020-piped.csv', 'w') as out_file:
    with open('../data/accre-jobs-2020.csv') as in_file:
        for line in in_file.readlines():
            out_file.write(line.replace(',','|', 12))

In [9]:
accre_jobs_piped = pd.read_csv('../data/accre-jobs-2020-piped.csv', sep="|")

In [10]:
accre_jobs_piped.head()

,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,STATE,NODELIST
0,15925210,treviso,arabella,122880Mn,65973.49M,13-18:00:00,13-18:00:28,1,24,production,0:0,COMPLETED,cn1531
1,15861126,treviso,arabella,122880Mn,67181.12M,13-18:00:00,12-14:50:56,1,24,production,0:0,COMPLETED,cn1441
2,15861125,treviso,arabella,122880Mn,69111.86M,13-18:00:00,13-18:00:20,1,24,production,0:0,COMPLETED,cn1464
3,16251645,treviso,arabella,122880Mn,65317.33M,13-18:00:00,12-03:50:32,1,24,production,0:0,COMPLETED,cn1473
4,16251646,treviso,arabella,122880Mn,65876.11M,13-18:00:00,13-18:00:03,1,24,production,0:0,COMPLETED,cn1440


In [11]:
accre_jobs_piped.loc[3460:3470]

,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,STATE,NODELIST
3460,17050901_91,winged,lavonda,4096Mn,669.61M,12:00:00,00:06:05,4,1,production,0:0,COMPLETED,"cn[449,463,911,913]"
3461,17050901_92,winged,lavonda,4096Mn,500.06M,12:00:00,00:02:25,1,4,production,0:0,COMPLETED,cn1336
3462,17050901_93,winged,lavonda,4096Mn,530.52M,12:00:00,00:02:19,1,4,production,0:0,COMPLETED,cn1319
3463,17050901_94,winged,lavonda,4096Mn,620.31M,12:00:00,00:05:59,3,1,production,0:0,COMPLETED,"cn[425,449,1134]"
3464,17050901_95,winged,lavonda,4096Mn,484.49M,12:00:00,00:02:08,1,4,production,0:0,COMPLETED,cn1319
3465,17050901_96,winged,lavonda,4096Mn,304.21M,12:00:00,00:02:16,1,4,production,0:0,COMPLETED,cn1336
3466,17050901_97,winged,lavonda,4096Mn,629.38M,12:00:00,00:06:24,4,1,production,0:0,COMPLETED,"cn[394,463,911,913]"
3467,17050901_98,winged,lavonda,4096Mn,561.52M,12:00:00,00:02:04,1,4,production,0:0,COMPLETED,cn1319
3468,17050901_99,winged,lavonda,4096Mn,636.64M,12:00:00,00:02:00,1,4,production,0:0,COMPLETED,cn1319
3469,17050901_100,winged,lavonda,4096Mn,522.07M,12:00:00,00:02:09,1,4,production,0:0,COMPLETED,cn1336


In [12]:
accre_jobs_piped.sort_values('USEDTIME', ascending=False)

,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,STATE,NODELIST
1622427,19735157,mickey,penny,126976Mn,7541.80M,20-20:00:00,9-23:34:43,1,24,cgw-dougherty1,0:0,COMPLETED,dougherty1
461600,17713864_49,fresh,demonte,24576Mn,14165.00M,10-00:00:00,9-23:27:11,1,2,production,0:0,COMPLETED,cn316
3610224,23765382,boysenberries,hortensia,120000Mn,40152.81M,13-22:00:00,9-23:11:50,1,8,production,0:0,COMPLETED,cn1240
3610223,23765380,boysenberries,hortensia,120000Mn,39362.95M,13-22:00:00,9-23:11:36,1,8,production,0:0,COMPLETED,cn1237
2814590,22318738,lychees,elba,30720Mn,1349.73M,14-00:00:00,9-22:57:28,1,2,production,0:0,COMPLETED,cn414
...,...,...,...,...,...,...,...,...,...,...,...,...,...
761966,18105129,cms,cmspilot,62193Mn,0,2-00:00:00,00:00:00,1,12,nogpfs,0:0,COMPLETED,ng1113
2014499,20598113,cms,uscmslocal,46071Mn,0,2-00:00:00,00:00:00,1,8,nogpfs,0:0,COMPLETED,ng518
2014179,20596308,cmsadmin,lcgadmin,4096Mn,0.09M,2-00:00:00,00:00:00,1,1,sam,0:0,COMPLETED,vm-cms-sam-pri
652666,17985724,cms,cmslocal,21875Mn,0,2-00:00:00,00:00:00,1,8,nogpfs,0:0,COMPLETED,ng1027


In [13]:
accre_jobs_piped.columns = [col_name.lower() for col_name in accre_jobs_piped.columns]

In [14]:
accre_jobs_piped.head()

,jobid,account,user,reqmem,usedmem,reqtime,usedtime,nodes,cpus,partition,exitcode,state,nodelist
0,15925210,treviso,arabella,122880Mn,65973.49M,13-18:00:00,13-18:00:28,1,24,production,0:0,COMPLETED,cn1531
1,15861126,treviso,arabella,122880Mn,67181.12M,13-18:00:00,12-14:50:56,1,24,production,0:0,COMPLETED,cn1441
2,15861125,treviso,arabella,122880Mn,69111.86M,13-18:00:00,13-18:00:20,1,24,production,0:0,COMPLETED,cn1464
3,16251645,treviso,arabella,122880Mn,65317.33M,13-18:00:00,12-03:50:32,1,24,production,0:0,COMPLETED,cn1473
4,16251646,treviso,arabella,122880Mn,65876.11M,13-18:00:00,13-18:00:03,1,24,production,0:0,COMPLETED,cn1440


In [15]:
def get_seconds(used_time_string):
    """calculate total second in job time"""
    days = 0
    day_break = used_time_string.split('-')
    if len(day_break) > 1:
        days = int(day_break[0])
        hms = day_break[1]
    else:
        hms = day_break[0]
    hours, mins, secs = [int(part) for part in hms.split(':')]
    total_seconds = secs + (mins * 60) + (hours * 3600) + (days * 86400)
    return total_seconds    

In [16]:
accre_jobs_piped['req_total_seconds'] = accre_jobs_piped['reqtime'].apply(get_seconds)

In [17]:
accre_jobs_piped.head()

,jobid,account,user,reqmem,usedmem,reqtime,usedtime,nodes,cpus,partition,exitcode,state,nodelist,req_total_seconds
0,15925210,treviso,arabella,122880Mn,65973.49M,13-18:00:00,13-18:00:28,1,24,production,0:0,COMPLETED,cn1531,1188000
1,15861126,treviso,arabella,122880Mn,67181.12M,13-18:00:00,12-14:50:56,1,24,production,0:0,COMPLETED,cn1441,1188000
2,15861125,treviso,arabella,122880Mn,69111.86M,13-18:00:00,13-18:00:20,1,24,production,0:0,COMPLETED,cn1464,1188000
3,16251645,treviso,arabella,122880Mn,65317.33M,13-18:00:00,12-03:50:32,1,24,production,0:0,COMPLETED,cn1473,1188000
4,16251646,treviso,arabella,122880Mn,65876.11M,13-18:00:00,13-18:00:03,1,24,production,0:0,COMPLETED,cn1440,1188000


In [18]:
accre_jobs_piped['used_total_seconds'] = accre_jobs_piped['usedtime'].apply(get_seconds)

In [19]:
accre_jobs_piped.head()

,jobid,account,user,reqmem,usedmem,reqtime,usedtime,nodes,cpus,partition,exitcode,state,nodelist,req_total_seconds,used_total_seconds
0,15925210,treviso,arabella,122880Mn,65973.49M,13-18:00:00,13-18:00:28,1,24,production,0:0,COMPLETED,cn1531,1188000,1188028
1,15861126,treviso,arabella,122880Mn,67181.12M,13-18:00:00,12-14:50:56,1,24,production,0:0,COMPLETED,cn1441,1188000,1090256
2,15861125,treviso,arabella,122880Mn,69111.86M,13-18:00:00,13-18:00:20,1,24,production,0:0,COMPLETED,cn1464,1188000,1188020
3,16251645,treviso,arabella,122880Mn,65317.33M,13-18:00:00,12-03:50:32,1,24,production,0:0,COMPLETED,cn1473,1188000,1050632
4,16251646,treviso,arabella,122880Mn,65876.11M,13-18:00:00,13-18:00:03,1,24,production,0:0,COMPLETED,cn1440,1188000,1188003


In [20]:
accre_jobs_piped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3816290 entries, 0 to 3816289
Data columns (total 15 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   jobid               object
 1   account             object
 2   user                object
 3   reqmem              object
 4   usedmem             object
 5   reqtime             object
 6   usedtime            object
 7   nodes               int64 
 8   cpus                int64 
 9   partition           object
 10  exitcode            object
 11  state               object
 12  nodelist            object
 13  req_total_seconds   int64 
 14  used_total_seconds  int64 
dtypes: int64(4), object(11)
memory usage: 436.7+ MB


In [21]:
def get_mb_per_core(row):
    if 'Mn' in row['reqmem']:
        cpus = row['cpus'] or 1
        reqmem = int(row['reqmem'][:-2]) / (row['nodes'] * cpus)
    elif 'Mc' in row['reqmem']:
        reqmem = int(row['reqmem'][:-2])
    else:
        reqmem = row['reqmem']
    return reqmem
        

In [22]:
accre_jobs_piped['mb_per_core'] = accre_jobs_piped.apply(get_mb_per_core, axis = 1)

In [23]:
accre_jobs_piped.head()

,jobid,account,user,reqmem,usedmem,reqtime,usedtime,nodes,cpus,partition,exitcode,state,nodelist,req_total_seconds,used_total_seconds,mb_per_core
0,15925210,treviso,arabella,122880Mn,65973.49M,13-18:00:00,13-18:00:28,1,24,production,0:0,COMPLETED,cn1531,1188000,1188028,5120.0
1,15861126,treviso,arabella,122880Mn,67181.12M,13-18:00:00,12-14:50:56,1,24,production,0:0,COMPLETED,cn1441,1188000,1090256,5120.0
2,15861125,treviso,arabella,122880Mn,69111.86M,13-18:00:00,13-18:00:20,1,24,production,0:0,COMPLETED,cn1464,1188000,1188020,5120.0
3,16251645,treviso,arabella,122880Mn,65317.33M,13-18:00:00,12-03:50:32,1,24,production,0:0,COMPLETED,cn1473,1188000,1050632,5120.0
4,16251646,treviso,arabella,122880Mn,65876.11M,13-18:00:00,13-18:00:03,1,24,production,0:0,COMPLETED,cn1440,1188000,1188003,5120.0


In [24]:
accre_jobs_piped['nodelist'][3460:3470].to_list()

['cn[449,463,911,913]',
 'cn1336',
 'cn1319',
 'cn[425,449,1134]',
 'cn1319',
 'cn1336',
 'cn[394,463,911,913]',
 'cn1319',
 'cn1319',
 'cn1336']

In [25]:
accre_jobs_piped['nodelist'][3460:3470].str.findall(r'(\D+)*(\d+)')

3460    [(cn[, 449), (,, 463), (,, 911), (,, 913)]
3461                                  [(cn, 1336)]
3462                                  [(cn, 1319)]
3463             [(cn[, 425), (,, 449), (,, 1134)]
3464                                  [(cn, 1319)]
3465                                  [(cn, 1336)]
3466    [(cn[, 394), (,, 463), (,, 911), (,, 913)]
3467                                  [(cn, 1319)]
3468                                  [(cn, 1319)]
3469                                  [(cn, 1336)]
Name: nodelist, dtype: object

In [26]:
accre_jobs_piped['node_index'] = accre_jobs_piped['nodelist'].str.extract(r'([a-z-]+)',expand=False)
accre_jobs_piped

,jobid,account,user,reqmem,usedmem,reqtime,usedtime,nodes,cpus,partition,exitcode,state,nodelist,req_total_seconds,used_total_seconds,mb_per_core,node_index
0,15925210,treviso,arabella,122880Mn,65973.49M,13-18:00:00,13-18:00:28,1,24,production,0:0,COMPLETED,cn1531,1188000,1188028,5120.0,cn
1,15861126,treviso,arabella,122880Mn,67181.12M,13-18:00:00,12-14:50:56,1,24,production,0:0,COMPLETED,cn1441,1188000,1090256,5120.0,cn
2,15861125,treviso,arabella,122880Mn,69111.86M,13-18:00:00,13-18:00:20,1,24,production,0:0,COMPLETED,cn1464,1188000,1188020,5120.0,cn
3,16251645,treviso,arabella,122880Mn,65317.33M,13-18:00:00,12-03:50:32,1,24,production,0:0,COMPLETED,cn1473,1188000,1050632,5120.0,cn
4,16251646,treviso,arabella,122880Mn,65876.11M,13-18:00:00,13-18:00:03,1,24,production,0:0,COMPLETED,cn1440,1188000,1188003,5120.0,cn
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3816285,24173815_8,portabella,vennie,32768Mn,30042.68M,1-00:00:00,00:02:27,1,2,production,0:0,COMPLETED,cn312,86400,147,16384.0,cn
3816286,24173815_9,portabella,vennie,32768Mn,31067.75M,1-00:00:00,00:02:27,1,2,production,0:0,COMPLETED,cn312,86400,147,16384.0,cn
3816287,24173815_10,portabella,vennie,32768Mn,21207.96M,1-00:00:00,00:01:40,1,2,production,0:0,COMPLETED,cn368,86400,100,16384.0,cn
3816288,24173817,boysenberries,hortensia,100000Mn,0,1-00:00:00,00:03:05,1,8,production,0:0,RUNNING,cn1509,86400,185,12500.0,cn


In [27]:
accre_jobs_piped['node_index'].unique()

array(['cn', 'dougherty', 'capra', 'cqs', 'gpu', 'tbi', 'ng',
       'vm-cms-sam-pri', 'vm-cms-sam-sec', 'eval-dell-', 'rocksteady',
       'vm-qa-flatearth'], dtype=object)

In [28]:
accre_jobs_piped['node_numbers'] = accre_jobs_piped['nodelist'].str.findall(r'(\d+)')
accre_jobs_piped

,jobid,account,user,reqmem,usedmem,reqtime,usedtime,nodes,cpus,partition,exitcode,state,nodelist,req_total_seconds,used_total_seconds,mb_per_core,node_index,node_numbers
0,15925210,treviso,arabella,122880Mn,65973.49M,13-18:00:00,13-18:00:28,1,24,production,0:0,COMPLETED,cn1531,1188000,1188028,5120.0,cn,[1531]
1,15861126,treviso,arabella,122880Mn,67181.12M,13-18:00:00,12-14:50:56,1,24,production,0:0,COMPLETED,cn1441,1188000,1090256,5120.0,cn,[1441]
2,15861125,treviso,arabella,122880Mn,69111.86M,13-18:00:00,13-18:00:20,1,24,production,0:0,COMPLETED,cn1464,1188000,1188020,5120.0,cn,[1464]
3,16251645,treviso,arabella,122880Mn,65317.33M,13-18:00:00,12-03:50:32,1,24,production,0:0,COMPLETED,cn1473,1188000,1050632,5120.0,cn,[1473]
4,16251646,treviso,arabella,122880Mn,65876.11M,13-18:00:00,13-18:00:03,1,24,production,0:0,COMPLETED,cn1440,1188000,1188003,5120.0,cn,[1440]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3816285,24173815_8,portabella,vennie,32768Mn,30042.68M,1-00:00:00,00:02:27,1,2,production,0:0,COMPLETED,cn312,86400,147,16384.0,cn,[312]
3816286,24173815_9,portabella,vennie,32768Mn,31067.75M,1-00:00:00,00:02:27,1,2,production,0:0,COMPLETED,cn312,86400,147,16384.0,cn,[312]
3816287,24173815_10,portabella,vennie,32768Mn,21207.96M,1-00:00:00,00:01:40,1,2,production,0:0,COMPLETED,cn368,86400,100,16384.0,cn,[368]
3816288,24173817,boysenberries,hortensia,100000Mn,0,1-00:00:00,00:03:05,1,8,production,0:0,RUNNING,cn1509,86400,185,12500.0,cn,[1509]


# Stan 
    Question  1    Which partition nodes show high number of failures 

# Jason --
    Question: 2   CMS Frequency of failure on particular nodes - user names: "cmslocal" and "cmspilot"

The CMS collaboration has an automated job submission system that runs jobs as "cmslocal" and "cmspilot". For these two users, jobs have internal system tests that will terminate their jobs early after approximately 30 minutes. Do any of their jobs that ended in under an hour also cluster on specific compute nodes, suggesting possbily unreliable systems? Check both “production” and “nogpfs” partitions. Look for commonly failing nodes and compare with other failed jobs.

In [29]:
# plt.style.use('dark_background')


In [30]:
cms_jobs_df = accre_jobs_piped[(accre_jobs_piped['user'] == "cmslocal") | (accre_jobs_piped['user'] == "cmspilot")]

In [31]:
total_jobs = cms_jobs_df.shape[0]

In [32]:
cms_jobs_df['failure'] = np.where(cms_jobs_df['exitcode'] == "0:0", 0, 1)

<ipython-input-32-ee7b4d49ef10>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cms_jobs_df['failure'] = np.where(cms_jobs_df['exitcode'] == "0:0", 0, 1)


In [33]:
cms_jobs_failed_df = cms_jobs_df[cms_jobs_df['failure'] == 1]

In [34]:
failures = cms_jobs_failed_df.shape[0]

In [35]:
failures / total_jobs * 100

0.027491310355378974

In [36]:
cms_jobs_df[cms_jobs_df['used_total_seconds'] < 3660].sort_values('used_total_seconds', ascending=False)

,jobid,account,user,reqmem,usedmem,reqtime,usedtime,nodes,cpus,partition,exitcode,state,nodelist,req_total_seconds,used_total_seconds,mb_per_core,node_index,node_numbers,failure
402503,17613549,cms,cmspilot,28000Mn,4745.66M,2-00:00:00,01:00:59,1,4,production,0:0,COMPLETED,cn1518,172800,3659,7000.000000,cn,[1518],0
3599782,23738991,cms,cmspilot,28000Mn,5793.68M,2-00:00:00,01:00:59,1,4,production,0:0,COMPLETED,cn408,172800,3659,7000.000000,cn,[408],0
3700442,23947055,cms,cmslocal,28000Mn,7649.01M,2-00:00:00,01:00:59,1,4,production,0:0,COMPLETED,cn392,172800,3659,7000.000000,cn,[392],0
1894786,20352599,cms,cmslocal,5000Mn,1787.50M,2-00:00:00,01:00:59,1,1,production,0:0,COMPLETED,cn452,172800,3659,5000.000000,cn,[452],0
274821,17447254,cms,cmslocal,186648Mn,69683.26M,2-00:00:00,01:00:59,1,32,nogpfs,0:0,COMPLETED,cn1549,172800,3659,5832.750000,cn,[1549],0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
700725,18040975,cms,cmslocal,38007Mn,0,2-00:00:00,00:00:00,1,8,nogpfs,0:0,COMPLETED,ng512,172800,0,4750.875000,ng,[512],0
700726,18040976,cms,cmslocal,42994Mn,0,2-00:00:00,00:00:00,1,12,nogpfs,0:0,COMPLETED,ng205,172800,0,3582.833333,ng,[205],0
700732,18040982,cms,cmspilot,21875Mn,0,2-00:00:00,00:00:00,1,8,nogpfs,0:0,COMPLETED,ng909,172800,0,2734.375000,ng,[909],0
700738,18040988,cms,cmspilot,38007Mn,0,2-00:00:00,00:00:00,1,8,nogpfs,0:0,COMPLETED,ng512,172800,0,4750.875000,ng,[512],0


In [37]:
early_exits_df = cms_jobs_df[cms_jobs_df['used_total_seconds'] < 1920]

In [38]:
early_exits = early_exits_df.shape[0]

In [39]:
early_exits

481093

In [40]:
early_exits /total_jobs * 100

57.50381292521885

In [41]:
node_early_exists = early_exits_df['nodelist'].value_counts()

In [42]:
node_jobs = cms_jobs_df['nodelist'].value_counts()

In [43]:
node_jobs_df = pd.concat([node_early_exists, node_jobs], axis=1)

In [44]:
node_jobs_df.columns = ['early_exits', 'jobs']

In [45]:
node_jobs_df['failure_percent'] = node_jobs_df['early_exits'] / node_jobs_df['jobs'] * 100

In [46]:
node_jobs_df['failure_percent'].describe()

count    818.000000
mean      50.708806
std       17.515431
min       14.285714
25%       39.908123
50%       45.521436
75%       53.057066
max      100.000000
Name: failure_percent, dtype: float64

In [47]:
high_failure_nodes = node_jobs_df[node_jobs_df['failure_percent']> 53.057066].index.to_list()

# Next Steps
- GO back and map high failure_nodes into the cms_jobs list and all the jobs and see if it dispropoiately fails.
- Map out failures by partition
- Map out failure clusters by order to job execution by order of job id

In [48]:
len(high_failure_nodes)

205

# Maggi -- 
    Question: 3 & 4   Calculation of % of actual memory used and mean % for each group